In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv'):
            os.remove(os.path.join(directory_path, file))
    csv_file = os.path.join(directory_path, "url_data.csv")

    
    combined_df.to_csv(csv_file, index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                title = title.replace("/", "_")
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

    return len(failed_downloads) == 0  # 모든 다운로드가 성공했으면 True 반환



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    script_dir = '/Users/jonathanoh/Desktop/script/crawling'
    duplicate_url = combine_csv_files_remove_duplicates(script_dir)
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urlsx = duplicate_url.loc[:, 'url'].to_list()
    old_urls = []
    for x in old_urlsx :
        old_urls.append(x.split('?')[0])

    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href.split('?')[0] in old_urls : 
                mm = href.split('?')[0].split('/')[-1]
                print(f"Processing: {total} 중 {i} {mm}__{title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href.split('?')[0], title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [3]:

download_path = '/Users/jonathanoh/Desktop/down'
excel_path = '/Users/jonathanoh/Desktop/down/image_url.xlsx'

options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')

In [1]:


# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

NameError: name 'fetch_post_list' is not defined

In [15]:
post_list = []
for i in range(3):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)
post_list    

[('https://arca.live/b/aiartreal/95074892?p=1',
  'SDXL+majicmix7,hanimix_real_v12,hanimix_real_v14,(face)',
  'sandy'),
 ('https://arca.live/b/aiartreal/95074503?p=1',
  'hanimix_XL_v11Au,hanimix_XL_v10B,++TPix_typeRE_v1.6(face)',
  'sandy'),
 ('https://arca.live/b/aiartreal/95071246?p=1',
  'AI 생성 이미지 판매 가능한 곳 몇군데',
  '젬마'),
 ('https://arca.live/b/aiartreal/95070370?p=1', '축제의 밤(6)', 'MS07B3'),
 ('https://arca.live/b/aiartreal/95069724?p=1',
  'NeMix Real-D v1.4.0 부먹...',
  'coffee7897'),
 ('https://arca.live/b/aiartreal/95069433?p=1', '유튜브 썸네일 좋아', '구슬사탕'),
 ('https://arca.live/b/aiartreal/95069286?p=1', 'NAI', 'laskjdjlasdf'),
 ('https://arca.live/b/aiartreal/95069262?p=1', '와! 복구!', 'Invalid_Syntax'),
 ('https://arca.live/b/aiartreal/95068665?p=1',
  'hanimix_real_v15B 찍먹해봅니다..',
  'coffee7897'),
 ('https://arca.live/b/aiartreal/95066759?p=1',
  '무지성프롬으로 찍먹해보는 하니믹스 15a+b',
  '나이트시티'),
 ('https://arca.live/b/aiartreal/95066077?p=1', 'dragon witch', '바코드'),
 ('https://arca.live/b/ai

In [56]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

중복 : SDXL+majicmix7,hanimix_real_v12,hanimix_real_v14,(face)
중복 : hanimix_XL_v11Au,hanimix_XL_v10B,++TPix_typeRE_v1.6(face)
중복 : AI 생성 이미지 판매 가능한 곳 몇군데
중복 : 축제의 밤(6)
중복 : NeMix Real-D v1.4.0 부먹...
중복 : 유튜브 썸네일 좋아
중복 : NAI
중복 : 와! 복구!
중복 : hanimix_real_v15B 찍먹해봅니다..
중복 : 무지성프롬으로 찍먹해보는 하니믹스 15a+b
중복 : dragon witch
중복 : 스트레스 없이 생산 활동 하고 있습니다.
중복 : (신출내기) 흡연하지 마시오.
중복 : ComfyUI) 인물 여러명 1명씩 detailer로 칠하기
중복 : 섹시한 메이드?
중복 : 뉴비가 만든 와인녀입니다
중복 : 유명 남성잡지 메인모델 B양...(스압)
중복 : 주딱이 되고나니
중복 : 자캐탄생
중복 : office
중복 : hanimix_real_v15A,B 냠냠
중복 : 너무나도 연악해보이는 일진녀
중복 : 상대 야쿠자 조직으로 혼자 쳐들어가 무쌍 찍는 여자 야쿠자 두목
중복 : hanimix_XL_v11Au + BMAB + 와카
중복 : 생초보:커피를 마시러 친구를 만나는 것
중복 : 일진녀
중복 : hanimix_XL_v11Ac, 찍먹 - 게임 캐릭터 컨셉
중복 : NeMix VurfY 찍먹과 부먹~
중복 : negligee
중복 : hanimix_real_v15A 찍먹
중복 : hanimix_real_v15B 테스트
중복 : knight
중복 : 청순......
중복 : 당분간 잠수 탑니다~~~
중복 : 역사 쇼츠를 포함해서 수면유도도 비추천
중복 : [NeMix F v2.8.0] 찍먹
중복 : lady of death
중복 : 에이프런 - 앞치마
중복 : blazers
중복 : (스압) cosplay New Jersey
중복 : Open Clothes, - 야발야발님 프롬 돚거
중복 

In [249]:
list = process_images(driver, list, download_path)

중복 : NeMix VurfY / 제한 공유
중복 : 소재) 식빵 출근 - 서술형 프롬 / NeMix F v2.8.0 찍먹
중복 : NeMix F v2.8.0 / 제한 공유
중복 : NeMix Real-D v1.4.0 / 제한 공유
중복 : military
중복 : 커미션 신청 할때 관련해서 궁금한게 있습니다 ( AI )
중복 : 토끼소녀
중복 : 1장) 오늘밤 당신의 xx을 훔쳐가겠습니다
중복 : sdxl 리파이너?
중복 : 에픽-고스트와이어 도쿄 배포
중복 : 알버트 휴보 로라 메일 다 보냈습니다
_____finished______
Failed posts:
No data available to create DataFrame


In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [55]:
temp = []
for url, title, user in list :
    temp.append([url, title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
script_dir = '/Users/jonathanoh/Desktop/script/crawling'
csv_fail_fiile = os.path.join(script_dir, 'fail_list.csv')
df.to_csv(csv_fail_fiile, index=False)
df = combine_csv_files_remove_duplicates(script_dir)

In [50]:
df = combine_csv_files_remove_duplicates(script_dir)

In [51]:
df

,url,filename
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_127497.jpeg
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_175719.jpeg
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_295008.jpeg
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_295538.jpeg
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Year_212334.jpeg
...,...,...
7225,https://arca.live/b/aiartreal/95051053?p=2,Cafe - 바코드님/머차가뜨님 돚거
7226,https://arca.live/b/aiartreal/95066061?p=1,스트레스 없이 생산 활동 하고 있습니다.
7227,https://arca.live/b/aiartreal/95053202?p=2,hanimix_real_v15B 셀프 찍먹
7228,https://arca.live/b/aiartreal/95052445?p=2,조선시대 로라말고 체크포인트도 있나요?


['94993242_젬마.zip',
 '94967708_리스펙트.zip',
 '95016252_TeamA.zip',
 '95042883_carmen.zip',
 '95043687_리버스Q.zip',
 '94976087_젬마.zip',
 'civitai_Day_20231225_070258.xlsx',
 '94958822_너예쁘다.zip',
 '95061365_호릉불.zip',
 '95060974_초콜릿주세요.zip',
 '94935829_리스펙트.zip',
 '95056954_ㅇㅇ.zip',
 '95000214_Gbox.zip',
 '95013973_야발야발.zip',
 '95069262_Invalid_Syntax.zip',
 '95057424_바코드.zip',
 '95068665_coffee7897.zip',
 '94987201_머리는차갑게가슴은뜨겁게.zip',
 '94973800_빠다파리.zip',
 '95056478_리버스Q.zip',
 '95001410_thec.zip',
 '95046524_스노우폭스.zip',
 '94965605_냥이집사.zip',
 '94977635_showmaker11.zip',
 '.DS_Store',
 '95042712_carmen.zip',
 '94995036_머리는차갑게가슴은뜨겁게.zip',
 '94942780_그림꾼달잡이.zip',
 '94958197_Ai우유.zip',
 '94995125_빠다파리.zip',
 '95057361_파르나.zip',
 '94996914_사랑은개나소나.zip',
 '94973976_빠다파리.zip',
 '94943294_빠다파리.zip',
 '94954341_리스펙트.zip',
 '95056949_머리는차갑게가슴은뜨겁게.zip',
 'url_data.csv',
 '94926326_그림꾼달잡이.zip',
 '95017531_야발야발.zip',
 '95023245_yong91892.zip',
 '95063411_그림꾼달잡이.zip',
 '94992489_하니타카.zip',
 '94975930

In [52]:
df['url'] = df['url'].apply(lambda x: x.split('?')[0])

In [53]:
df.to_csv(os.path.join(script_dir, "url_datas.csv"), index=False)

<bound method NDFrame.head of                                                     url  \
0     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
1     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
2     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
3     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
4     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
...                                                 ...   
7226             https://arca.live/b/aiartreal/95066061   
7227             https://arca.live/b/aiartreal/95053202   
7228             https://arca.live/b/aiartreal/95052445   
7229             https://arca.live/b/aiartreal/95074503   
9718             https://arca.live/b/aiartreal/95074892   

                                               filename  
0                              civitai_Year_127497.jpeg  
1                              civitai_Year_175719.jpeg  
2                              civitai_Year_295008.jpeg  
3                            